# Python 知识

# 代码设计

## 类继承关系

```mermaid
classDiagram
    %% 元类层次结构
    class asyncio.CancelledError
    class Job
    class Scheduler
    class CancelledError
    class AsyncJob
    class AsyncScheduler
    class ScheduleManager
    
    %% 继承关系
    asyncio.CancelledError <|-- CancelledError
    Job <|-- AsyncJob
    Scheduler <|-- AsyncScheduler
```
异步任务调度管理模块，主要用于：
1. 定期数据获取和更新（如股价数据、新闻数据等）
2. 策略定时执行（如每日交易信号生成）
3. 报告和监控任务（如风险监控、绩效报告）
4. 系统维护任务（如数据清理、缓存更新）

## class CancelledError(asyncio.CancelledError)

```python
class CancelledError(asyncio.CancelledError):
    pass
```

## class AsyncJob(Job)

```python
class AsyncJob(Job):
    async def async_run(self) -> tp.Any:
        logger.info('Running job %s', self)
        ret = self.job_func()
        if inspect.isawaitable(ret):
            ret = await ret
        self.last_run = datetime.now()
        self._schedule_next_run()
        return ret
```

## class AsyncScheduler(Scheduler)

```python
class AsyncScheduler(Scheduler):
    async def async_run_pending(self) -> None:
        runnable_jobs = (job for job in self.jobs if job.should_run)
        await asyncio.gather(*[self._async_run_job(job) for job in runnable_jobs])

    async def async_run_all(self, delay_seconds: int = 0) -> None:
        logger.info('Running *all* %i jobs with %is delay in-between',
                    len(self.jobs), delay_seconds)
        for job in self.jobs[:]:
            await self._async_run_job(job)
            await asyncio.sleep(delay_seconds)

    async def _async_run_job(self, job: AsyncJob) -> None:
        ret = await job.async_run()
        if isinstance(ret, CancelJob) or ret is CancelJob:
            self.cancel_job(job)

    def every(self, interval: int = 1) -> AsyncJob:
        job = AsyncJob(interval, self)
        return job
```

## ScheduleManager

```python
class ScheduleManager:

    units: tp.ClassVar[tp.Tuple[str, ...]] = (
        "second",
        "seconds",
        "minute",
        "minutes",
        "hour",
        "hours",
        "day",
        "days",
        "week",
        "weeks"
    )

    weekdays: tp.ClassVar[tp.Tuple[str, ...]] = (
        "monday",
        "tuesday",
        "wednesday",
        "thursday",
        "friday",
        "saturday",
        "sunday",
    )

    def __init__(self, scheduler: tp.Optional[AsyncScheduler] = None) -> None:
        if scheduler is None:
            scheduler = AsyncScheduler()
        checks.assert_instance_of(scheduler, AsyncScheduler)

        self._scheduler = scheduler
        self._async_task = None

    @property
    def scheduler(self) -> AsyncScheduler:
        return self._scheduler

    @property
    def async_task(self) -> tp.Optional[asyncio.Task]:
        return self._async_task

    def every(self, *args, to: tp.Optional[int] = None,
              tags: tp.Optional[tp.Iterable[tp.Hashable]] = None) -> AsyncJob:
        # Parse arguments
        interval = 1
        unit = None
        start_day = None
        at = None

        def _is_arg_interval(arg):
            return isinstance(arg, (int, timedelta))

        def _is_arg_unit(arg):
            return isinstance(arg, str) and arg in self.units

        def _is_arg_start_day(arg):
            return isinstance(arg, str) and arg in self.weekdays

        def _is_arg_at(arg):
            return (isinstance(arg, str) and ':' in arg) or isinstance(arg, dt_time)

        expected_args = ['interval', 'unit', 'start_day', 'at']
        for i, arg in enumerate(args):
            if 'interval' in expected_args and _is_arg_interval(arg):
                interval = arg
                expected_args = expected_args[expected_args.index('interval') + 1:]
                continue
            if 'unit' in expected_args and _is_arg_unit(arg):
                unit = arg
                expected_args = expected_args[expected_args.index('unit') + 1:]
                continue
            if 'start_day' in expected_args and _is_arg_start_day(arg):
                start_day = arg
                expected_args = expected_args[expected_args.index('start_day') + 1:]
                continue
            if 'at' in expected_args and _is_arg_at(arg):
                at = arg
                expected_args = expected_args[expected_args.index('at') + 1:]
                continue
            raise ValueError(f"Arg at index {i} is unexpected")

        if at is not None:
            if unit is None and start_day is None:
                unit = 'days'
        if unit is None and start_day is None:
            unit = 'seconds'

        job = self.scheduler.every(interval)
        if unit is not None:
            job = getattr(job, unit)
        if start_day is not None:
            job = getattr(job, start_day)
        if at is not None:
            if isinstance(at, dt_time):
                if job.unit == "days" or job.start_day:
                    if at.tzinfo is not None:
                        at = tzaware_to_naive_time(at, None)
                at = at.isoformat()
                if job.unit == "hours":
                    at = ':'.join(at.split(':')[1:])
                if job.unit == "minutes":
                    at = ':' + at.split(':')[2]
            job = job.at(at)
        if to is not None:
            job = job.to(to)
        if tags is not None:
            if not isinstance(tags, tuple):
                tags = (tags,)
            job = job.tag(*tags)

        return job

    def start(self, sleep: int = 1) -> None:
        logger.info("Starting schedule manager with jobs %s", str(self.scheduler.jobs))
        try:
            while True:
                self.scheduler.run_pending()
                time.sleep(sleep)
        except (KeyboardInterrupt, asyncio.CancelledError):
            logger.info("Stopping schedule manager")

    async def async_start(self, sleep: int = 1) -> None:
        logger.info("Starting schedule manager in the background with jobs %s", str(self.scheduler.jobs))
        logger.info("Jobs: %s", str(self.scheduler.jobs))
        try:
            while True:
                await self.scheduler.async_run_pending()
                await asyncio.sleep(sleep)
        except asyncio.CancelledError:
            logger.info("Stopping schedule manager")

    def done_callback(self, async_task: asyncio.Task) -> None:
        logger.info(async_task)

    def start_in_background(self, **kwargs) -> None:
        async_task = asyncio.create_task(self.async_start(**kwargs))
        async_task.add_done_callback(self.done_callback)
        logger.info(async_task)
        self._async_task = async_task

    @property
    def async_task_running(self) -> bool:
        return self.async_task is not None and not self.async_task.done()

    def stop(self) -> None:
        if self.async_task_running:
            self.async_task.cancel()
```